In [9]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.llms import Ollama
from langgraph.graph import StateGraph, START, END

llm = Ollama(
    model="deepseek-r1:8b",
    temperature=0.7
)

class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    # Get the current messages from the state
    current_messages = state['messages']
    
    # Get the latest message (usually human message) to respond to
    latest_message = current_messages[-1]
    
    # Generate response
    # response = llm.invoke([latest_message])
    response = llm.invoke(current_messages)
    
    # Create a new AIMessage
    ai_message = AIMessage(content=response)
    
    # Create a new state with the updated messages
    new_state = {
        'messages': current_messages + [ai_message]
    }
    
    print(f"\nAI: {response}")
    print("CURRENT STATE: ", new_state["messages"])
    return new_state

# Build the graph
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)

agent = graph.compile()

# Instead of resetting the entire state, let's maintain a conversation history properly
conversation_history = []

user_input = input("Enter: ")

while user_input != "exit":
    # Add user message to history
    conversation_history.append(HumanMessage(content=user_input))
    
    # Update the state with the new user message
    state_input = {'messages': conversation_history}
    
    # Invoke the agent
    result = agent.invoke(state_input)
    
    # Get the updated conversation history from the result
    conversation_history = result['messages']
    
    user_input = input("Enter something: ")

# Save conversation with UTF-8 encoding
with open("logging.txt", "w", encoding='utf-8') as file:
    file.write("Your conversation Log:\n")
    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n")
    file.write("End of conversation")

print("conversation saved to logging.txt")


AI: Hi Abdelaziz! That's a great name! 😊 And awesome that you like basketball—it's such an exciting and fun sport! Do you have a favorite team or player? Let me know if you want to chat about the game or upcoming games!
CURRENT STATE:  [HumanMessage(content='hi I am abdelaziz And I like basketball', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Abdelaziz! That's a great name! 😊 And awesome that you like basketball—it's such an exciting and fun sport! Do you have a favorite team or player? Let me know if you want to chat about the game or upcoming games!", additional_kwargs={}, response_metadata={})]

AI: Your name is **Abdelaziz**, and your favorite sport is **basketball**! 😊
CURRENT STATE:  [HumanMessage(content='hi I am abdelaziz And I like basketball', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Abdelaziz! That's a great name! 😊 And awesome that you like basketball—it's such an exciting and fun sport! Do you have a favorite team or player